In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../fashion_data/processed_data/user_test.json'
user_valid_path = '../fashion_data/processed_data/user_valid.json'
user_train_path = '../fashion_data/processed_data/user_train.json'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
print(len(user_train_dict))

1273


In [4]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

1273


In [5]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

1273


In [6]:
print(user_valid_dict['0'])
print(user_test_dict['0'])

[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]


In [7]:
item_information_path = '../fashion_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict), item_information_dict['1'])

6089 {'title': 'Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M', 'brand': 'Allegra K', 'description': '\n\nThe product is a comfortable, stylish, and soft top, available in a great color. However, it has issues with sizing, fitting thin arms, and not being suitable for leggings.'}


In [8]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [9]:
user_profile_summary_path = '../fashion_data/processed_data/user_profile_summary.json'
with open(user_profile_summary_path, 'r') as f:
    user_profile_summary_dict = json.load(f)
print(len(user_profile_summary_dict), user_profile_summary_dict['1'])

848 
The user seems to prefer stylish and high-quality products, with a preference for good design and functionality. They liked the Destiny-themed wallet for its cool design and good quality, but disliked the hard-to-open button. They appreciated the spacious and sturdy backpack, but noted that it may not hold too much weight or size. The user liked the formal pencil dress for its stylish and expensive look, but found it unsuitable for pear-shaped women. Overall, the user values both style and functionality in their fashion purchases.


### Creating zeroshot data

In [10]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    if 'brand' in item_information:
        item_details += f" Brand: {item_information['brand']}"
    if 'price' in item_information:
        item_details += f" Price: {item_information['price']}"
    if 'description' in item_information:
        item_details += f" Description: {item_information['description']}"
    return item_details

In [11]:
### Test Data
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_test_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    if user in user_profile_summary_dict:
        prompt = f"""You are an expert fashion product recommender. You are provided with a user's profile and list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        prompt = """You are an expert fashion product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        """
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert fashion product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{target_item_details}" in atmost 100 words."""
    zeroshot_dict[user] = prompt
    # break

In [12]:
### Valid Data
zeroshot_valid_dict = dict()
check_missed_items = []
for user, item_list in user_valid_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    if user in user_profile_summary_dict:
        prompt = f"""You are an expert fashion product recommender. You are provided with a user's profile and list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        prompt = """You are an expert fashion product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        """
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert fashion product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{target_item_details}" in atmost 100 words."""
    zeroshot_valid_dict[user] = prompt
    # break

In [13]:
print(len(zeroshot_dict), len(zeroshot_valid_dict))

1273 1273


In [14]:
print(zeroshot_dict['0'])
print("-"*100)
print(zeroshot_valid_dict['0'])

You are an expert fashion product recommender. You are provided with a user's profile and list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User Profile - 

The user seems to prefer long-sleeved shirts and knit fabric, as indicated by their purchase of the Allegra K Lady Long Sleeve Letter Pattern Pullover Knit Shirt Black S. They appreciate a good quality fabric and length, but also value comfort and simplicity. The user disliked the product because it was too casual, suggesting they may prefer more formal or stylish clothing. The user's preference for leggings as a pairing item also implies they may enjoy a more fitted or form-fitting style. Overall, the user values comfort, quality, and style, with a preference for long-sleeved knit shirts and fitted styles.
        User purchased the following items in the given order. List of recent items and their description -
        Liked Allegra K Woman Boat Neck Butto

In [15]:
total_len = 0
for user, prompt in zeroshot_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

508.27101335428125


In [16]:
users_very_long = []
for user, prompt in zeroshot_dict.items():
    if len(prompt.strip().split(' ')) > 700:
        users_very_long.append(user)
print(len(users_very_long))

19


In [17]:
for key in users_very_long:
    zeroshot_dict.pop(key, None)
print(len(zeroshot_dict))

1254


In [18]:
total_len = 0
for user, prompt in zeroshot_valid_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

1406.518341307815


In [22]:
users_very_long = []
for user, prompt in zeroshot_valid_dict.items():
    if len(prompt.strip().split(' ')) > 700:
        users_very_long.append(user)
print(len(users_very_long))

339


In [23]:
for key in users_very_long:
    zeroshot_valid_dict.pop(key, None)
print(len(zeroshot_valid_dict))

934


In [24]:
zeroshot_valid_path = './ctr_zeroshot_dataset/zeroshot_valid.json'
with open(zeroshot_valid_path, 'w+') as f:
    json.dump(zeroshot_valid_dict, f)

In [25]:
zeroshot_test_path = './ctr_zeroshot_dataset/zeroshot_test.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [26]:
ratings_df[ratings_df['user'] == int('0')]

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400
5,5,0,3.0,1453766400
6,6,0,2.0,1453766400
7,7,0,3.0,1500854400


In [27]:
test_prompt = zeroshot_dict['34']
print(test_prompt)
print("-"*100)
response = openai.completions.create(
    # model = "gpt-3.5-turbo-0125",
    model="gpt-3.5-turbo-instruct",
    prompt = test_prompt,
    temperature=0.3,
    max_tokens=256,
    # top_p=0.3,
    # frequency_penalty=0.5,
    # presence_penalty=0.5
)
print(response.choices[0].text)

You are an expert fashion product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        Liked Shoes Insoles From SHINAP: Comfort Shoes Inserts, Cut to Size, Lightweight & Breathable, Moisture Wicking Shoe Inserts. Perfect for All Shoes & All Sports, Athletics, Work, Running, Hiking! Brand: ShiNap Description: 

Product: Knee and Foot Pain Relief Inserts

Strengths: 
1. Works as advertised
2. Eased pain on knees and feet
3. Comfortable
4. Can fit to any shoes

Weaknesses: None mentioned in the
Liked Ysiop Mens Polyester Skinny Neck ties Bowtie Pocket Square 3pcs Set Brand: Ysiop Description: 

Product Description:
The product is a skinny tie with polka dots, having impressive color patterns and feel of material. It has a good overall construction.
Liked Niceshop Round B